In [1]:
from iotbx.map_manager import map_manager as MapManager
from mmtbx.model import manager as ModelManager
from iotbx.data_manager import DataManager
from iotbx.map_model_manager import map_model_manager as MapModelManager

import os
import pickle
import inspect

In [2]:
from phenix.ligands_cc.ligands_cc import LigandsCC, LigandSelection

In [3]:
# test defaults only
resname = "MG"
ls = LigandSelection(include_resnames=None,
                     include_only_resnames=None,
                     exclude_resnames=None,
                     include_residue_classes=None,
                     exclude_residue_classes=None,
                     )
assert(ls.query(resname) == False)

In [4]:


resname = "ADP"
ls = LigandSelection(include_resnames=None,
                     include_only_resnames=None,
                     exclude_resnames=None,
                     include_residue_classes=None,
                     exclude_residue_classes=None,
                     )
assert(ls.query(resname)== True)



In [5]:
# test include only

resname = "MG"
ls = LigandSelection(include_resnames=None,
                     include_only_resnames=["ATP"],
                     exclude_resnames=None,
                     include_residue_classes=None,
                     exclude_residue_classes=None,
                     )
assert(ls.query(resname)== False)

resname = "ATP"
ls = LigandSelection(include_resnames=None,
                     include_only_resnames=["ATP"],
                     exclude_resnames=None,
                     include_residue_classes=None,
                     exclude_residue_classes=None,
                     )
assert(ls.query(resname)==True)



In [6]:
# test supplied include_resnames
resname = "MG"
ls = LigandSelection(include_resnames=["MG"],
                     include_only_resnames=None,
                     exclude_resnames=None,
                     include_residue_classes=None,
                     exclude_residue_classes=None,
                     )
assert(ls.query(resname) == True)

resname = "ATP"
ls = LigandSelection(include_resnames=["MG"],
                     include_only_resnames=None,
                     exclude_resnames=None,
                     include_residue_classes=None,
                     exclude_residue_classes=None,
                     )
assert(ls.query(resname) == True)

In [7]:
# test supplied exclude_resnames
resname = "ATP"
ls = LigandSelection(include_resnames=None,
                     include_only_resnames=None,
                     exclude_resnames=["ATP"],
                     include_residue_classes=None,
                     exclude_residue_classes=None,
                     )
assert(ls.query(resname) == False)

In [8]:
# test supplied include_residue_classes
resname = "MG"
ls = LigandSelection(include_resnames=None,
                     include_only_resnames=None,
                     exclude_resnames=None,
                     include_residue_classes=["common_element"],
                     exclude_residue_classes=None)
assert(ls.query(resname) == True)

In [9]:
# test supplied exclude_residue_classes
resname = "ATP"
ls = LigandSelection(include_resnames=None,
                     include_only_resnames=None,
                     exclude_resnames=None,
                     include_residue_classes=None,
                     exclude_residue_classes=["other"])
assert(ls.query(resname) == False)

In [10]:
%%timeit
LigandSelection(include_resnames=None,
                        include_only_resnames=None,
                        exclude_resnames=None,
                        include_residue_classes=None,
                        exclude_residue_classes=["other"])

The slowest run took 7.76 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 891 ns per loop


## Get data from disk

In [11]:
# first get data from disk
model_path = "/home/user/ligands_cc_demo/data/6f1u_4169/6f1u_4169.cif"
map_path = "/home/user/ligands_cc_demo/data/6f1u_4169/6f1u_4169.map"



dm = DataManager()
dm.process_model_file(model_path)
dm.process_real_map_file(map_path)

Could not import the compiled Python-sander interface. Make sure you have the Python development libraries installed and that you have sourced amber.sh or amber.csh

## Get ligands out of a model

In [12]:
mmm = dm.get_map_model_manager()

In [13]:
ligands_cc = LigandsCC(mmm,resolution=3.4)

In [14]:
ligands_cc.ccmask_full_model

In [15]:
ligands_cc.process_ligands()

In [19]:
os.getcwd()

'/home/user/Dropbox/Phenix Start/CryoEM-ReRefine-Ligands/Notebooks'

In [24]:
# write ouput
output_dir = "./"

for mmmlig in ligands_cc.ligand_map_model_managers:
    ligand_name = LigandsCC.ligand_name(mmmlig.model())
    ligand_model_file = os.path.join(output_dir,ligand_name)
    ligand_map_file = os.path.join(output_dir,ligand_name+".map")
    
    dm.write_model_file(mmmlig.model(),
                          filename=ligand_model_file,
                          extension="cif",
                          overwrite=True)
    
    dm.write_real_map_file(mmmlig.map_manager(),
                          filename=ligand_map_file,
                          overwrite=True)

In [25]:
ligands_cc.validate()   

NameError: name 'mmmlig' is not defined

## Get ligand density maps and calculate CCmask

In [9]:
map_manager = dm.get_real_map()
map_manager.set_resolution(entry.resolution)
map_model_manager = MapModelManager(map_manager=map_manager,model=dm.get_model())

ligand_dict, ccmask_values = ligands_cc_from_mmm(map_model_manager)

Found 3 ligands in the model 


In [10]:
print(ccmask_values)

[0.820752074164057, 0.8099958071989116, 0.8403064511105527]


## Optionally write to disk

In [12]:
ligand_dict, ccmask_values = ligands_cc_from_mmm(map_model_manager,
                                                 output_directory="../data/")

Found 3 ligands in the model 
Writing model file: /net/cci-filer3/home/cschlick/CryoEM-ReRefine-Ligands/data/ADP_D.map
Writing map file: /net/cci-filer3/home/cschlick/CryoEM-ReRefine-Ligands/data/ADP_D.cif
Writing model file: /net/cci-filer3/home/cschlick/CryoEM-ReRefine-Ligands/data/ADP_F.map
Writing map file: /net/cci-filer3/home/cschlick/CryoEM-ReRefine-Ligands/data/ADP_F.cif
Writing model file: /net/cci-filer3/home/cschlick/CryoEM-ReRefine-Ligands/data/ADP_B.map
Writing map file: /net/cci-filer3/home/cschlick/CryoEM-ReRefine-Ligands/data/ADP_B.cif
